In [1]:
import pandas as pd
import numpy as np 
import re
import nltk 
import string
import spacy
from nltk.stem.porter import PorterStemmer
nltk.download('stopwords')
nltk.download('punkt')
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize

from sklearn.model_selection import train_test_split
from nltk.stem import PorterStemmer,WordNetLemmatizer
from nltk.util import ngrams



C:\Users\PC\anaconda3\lib\site-packages\scipy\__init__.py:146: UserWarning: A NumPy version >=1.16.5 and <1.23.0 is required for this version of SciPy (detected version 1.26.2
  warnings.warn(f"A NumPy version >={np_minversion} and <{np_maxversion}"
[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\PC\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package punkt to
[nltk_data]     C:\Users\PC\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt is already up-to-date!


In [2]:
#news_data=pd.read_csv(r'C:\Users\PC\data.csv')
#news_data.head()

#fake news ----> 1
#true news----->0
#news_data.shape

# counting the number of missing values in the dataset
#news_data.isnull().sum()

# replacing the null values with empty string
#news_data = news_data.fillna('')

# To select a single column by its name
#txt =news_data['Body']  








# data analysis

In [3]:
df=pd.read_csv('data.csv')

In [4]:
df.head()

,URLs,Headline,Body,Label
0,http://www.bbc.com/news/world-us-canada-414191...,Four ways Bob Corker skewered Donald Trump,Image copyright Getty Images\nOn Sunday mornin...,1
1,https://www.reuters.com/article/us-filmfestiva...,Linklater's war veteran comedy speaks to moder...,"LONDON (Reuters) - “Last Flag Flying”, a comed...",1
2,https://www.nytimes.com/2017/10/09/us/politics...,Trump’s Fight With Corker Jeopardizes His Legi...,The feud broke into public view last week when...,1
3,https://www.reuters.com/article/us-mexico-oil-...,Egypt's Cheiron wins tie-up with Pemex for Mex...,MEXICO CITY (Reuters) - Egypt’s Cheiron Holdin...,1
4,http://www.cnn.com/videos/cnnmoney/2017/10/08/...,Jason Aldean opens 'SNL' with Vegas tribute,"Country singer Jason Aldean, who was performin...",1


# Preprocessing

In [5]:
df.isnull().sum()

URLs         0
Headline     0
Body        21
Label        0
dtype: int64

In [6]:
df = df.fillna(' ')

In [7]:
df.isnull().sum()

URLs        0
Headline    0
Body        0
Label       0
dtype: int64

In [8]:
df.shape

(4009, 4)

In [9]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 4009 entries, 0 to 4008
Data columns (total 4 columns):
 #   Column    Non-Null Count  Dtype 
---  ------    --------------  ----- 
 0   URLs      4009 non-null   object
 1   Headline  4009 non-null   object
 2   Body      4009 non-null   object
 3   Label     4009 non-null   int64 
dtypes: int64(1), object(3)
memory usage: 125.4+ KB


In [10]:
df=df.fillna('')

In [11]:
df.isnull().sum()

URLs        0
Headline    0
Body        0
Label       0
dtype: int64

In [12]:
df.columns

Index(['URLs', 'Headline', 'Body', 'Label'], dtype='object')

In [13]:
#body= text headline + text body 
df['Body']=df['Headline']+''+df['Body']

In [14]:
#drop 2 columns -----> URL + Headline 
df=df.drop(['URLs','Headline'], axis=1)

In [15]:
df.head()

,Body,Label
0,Four ways Bob Corker skewered Donald TrumpImag...,1
1,Linklater's war veteran comedy speaks to moder...,1
2,Trump’s Fight With Corker Jeopardizes His Legi...,1
3,Egypt's Cheiron wins tie-up with Pemex for Mex...,1
4,Jason Aldean opens 'SNL' with Vegas tributeCou...,1


In [16]:
x=df['Body']#.values

In [17]:
y=df['Label']#.valyes

In [18]:
print(x)

0       Four ways Bob Corker skewered Donald TrumpImag...
1       Linklater's war veteran comedy speaks to moder...
2       Trump’s Fight With Corker Jeopardizes His Legi...
3       Egypt's Cheiron wins tie-up with Pemex for Mex...
4       Jason Aldean opens 'SNL' with Vegas tributeCou...
                              ...                        
4004    Trends to WatchTrends to Watch\n% of readers t...
4005    Trump Jr. Is Soon To Give A 30-Minute Speech F...
4006         Ron Paul on Trump, Anarchism & the AltRight 
4007    China to accept overseas trial data in bid to ...
4008    Vice President Mike Pence Leaves NFL Game Beca...
Name: Body, Length: 4009, dtype: object


In [19]:
print(y)

0       1
1       1
2       1
3       1
4       1
       ..
4004    0
4005    0
4006    0
4007    1
4008    0
Name: Label, Length: 4009, dtype: int64


In [20]:
y.shape

(4009,)

In [21]:
x.shape

(4009,)

In [22]:
text_column = 'Body'

In [23]:
text_data = df[text_column]

# tokenization 




In [24]:
print(text_data)

0       Four ways Bob Corker skewered Donald TrumpImag...
1       Linklater's war veteran comedy speaks to moder...
2       Trump’s Fight With Corker Jeopardizes His Legi...
3       Egypt's Cheiron wins tie-up with Pemex for Mex...
4       Jason Aldean opens 'SNL' with Vegas tributeCou...
                              ...                        
4004    Trends to WatchTrends to Watch\n% of readers t...
4005    Trump Jr. Is Soon To Give A 30-Minute Speech F...
4006         Ron Paul on Trump, Anarchism & the AltRight 
4007    China to accept overseas trial data in bid to ...
4008    Vice President Mike Pence Leaves NFL Game Beca...
Name: Body, Length: 4009, dtype: object


In [25]:
msg = df.copy()
msg.head(10)

,Body,Label
0,Four ways Bob Corker skewered Donald TrumpImag...,1
1,Linklater's war veteran comedy speaks to moder...,1
2,Trump’s Fight With Corker Jeopardizes His Legi...,1
3,Egypt's Cheiron wins tie-up with Pemex for Mex...,1
4,Jason Aldean opens 'SNL' with Vegas tributeCou...,1
5,JetNation FanDuel League; Week 4JetNation FanD...,0
6,Kansas Tried a Tax Plan Similar to Trump’s. It...,1
7,"India RBI chief: growth important, but not at ...",1
8,EPA chief to sign rule on Clean Power Plan exi...,1
9,Talks on sale of Air Berlin planes to easyJet ...,1


In [26]:
#rying to reset the index of a DataFrame named msg 
#and then display the first 10 rows using head(10).
msg.reset_index(inplace = True)
msg.head(10)

,index,Body,Label
0,0,Four ways Bob Corker skewered Donald TrumpImag...,1
1,1,Linklater's war veteran comedy speaks to moder...,1
2,2,Trump’s Fight With Corker Jeopardizes His Legi...,1
3,3,Egypt's Cheiron wins tie-up with Pemex for Mex...,1
4,4,Jason Aldean opens 'SNL' with Vegas tributeCou...,1
5,5,JetNation FanDuel League; Week 4JetNation FanD...,0
6,6,Kansas Tried a Tax Plan Similar to Trump’s. It...,1
7,7,"India RBI chief: growth important, but not at ...",1
8,8,EPA chief to sign rule on Clean Power Plan exi...,1
9,9,Talks on sale of Air Berlin planes to easyJet ...,1


In [27]:
msg['Body'][10]

'U.S. President Donald Trump Quietly Signs Law To Allow Warrantless Searches In Parts Of VA, DC and MDBy Aaron Kesel\nAs former White House chief of staff and current Chicago Mayor Rahm Emanuel has said: “You never let a serious crisis go to waste.” While everyone was worried about the hurricanes, U.S. President Donald Trump signed away the Fourth Amendment buried in a new bill. He set a precedent by empowering the police state to allow warrantless searches in parts of VA, MD and D.C. near the Metro train system.\nThe government can now openly enter and search private property without a warrant in parts of Virginia, Maryland and D.C.\nThe House Joint Resolution 76 was signed into law on Tuesday, Aug. 22, by President Trump. The text is rather boring, starting off with: “Granting the consent and approval of Congress for the Commonwealth of Virginia, the State of Maryland, and the District of Columbia to enter into a compact relating to the establishment of the Washington Metrorail Safet

In [28]:
txt=msg['Body'][10]

In [29]:
len(txt)

4073

In [30]:
len(set (txt))# number of vocabs without repetition

74

In [31]:
len(txt)# number of char

4073

In [32]:
tokens=word_tokenize(txt)

In [33]:
print(tokens)

['U.S.', 'President', 'Donald', 'Trump', 'Quietly', 'Signs', 'Law', 'To', 'Allow', 'Warrantless', 'Searches', 'In', 'Parts', 'Of', 'VA', ',', 'DC', 'and', 'MDBy', 'Aaron', 'Kesel', 'As', 'former', 'White', 'House', 'chief', 'of', 'staff', 'and', 'current', 'Chicago', 'Mayor', 'Rahm', 'Emanuel', 'has', 'said', ':', '“', 'You', 'never', 'let', 'a', 'serious', 'crisis', 'go', 'to', 'waste.', '”', 'While', 'everyone', 'was', 'worried', 'about', 'the', 'hurricanes', ',', 'U.S.', 'President', 'Donald', 'Trump', 'signed', 'away', 'the', 'Fourth', 'Amendment', 'buried', 'in', 'a', 'new', 'bill', '.', 'He', 'set', 'a', 'precedent', 'by', 'empowering', 'the', 'police', 'state', 'to', 'allow', 'warrantless', 'searches', 'in', 'parts', 'of', 'VA', ',', 'MD', 'and', 'D.C.', 'near', 'the', 'Metro', 'train', 'system', '.', 'The', 'government', 'can', 'now', 'openly', 'enter', 'and', 'search', 'private', 'property', 'without', 'a', 'warrant', 'in', 'parts', 'of', 'Virginia', ',', 'Maryland', 'and', 'D

# remove stopwords  and  punctuation from txt 

In [34]:
# Get English stop words
stop_words = set(stopwords.words('english'))


In [35]:
# Remove stop words from txt
filtered_txt = [word for word in tokens if word.lower() not in stop_words]
filtered_txt = [word for word in tokens if word.lower() not in string.punctuation]

In [36]:
# Print the ________original and ______filtered text
print("Original Text:")
print(txt)
print("\nText after removing stop words:")
print(filtered_txt)

Original Text:
U.S. President Donald Trump Quietly Signs Law To Allow Warrantless Searches In Parts Of VA, DC and MDBy Aaron Kesel
As former White House chief of staff and current Chicago Mayor Rahm Emanuel has said: “You never let a serious crisis go to waste.” While everyone was worried about the hurricanes, U.S. President Donald Trump signed away the Fourth Amendment buried in a new bill. He set a precedent by empowering the police state to allow warrantless searches in parts of VA, MD and D.C. near the Metro train system.
The government can now openly enter and search private property without a warrant in parts of Virginia, Maryland and D.C.
The House Joint Resolution 76 was signed into law on Tuesday, Aug. 22, by President Trump. The text is rather boring, starting off with: “Granting the consent and approval of Congress for the Commonwealth of Virginia, the State of Maryland, and the District of Columbia to enter into a compact relating to the establishment of the Washington Metr

# stemming  
#note stemming is NLP techniq used to reduce words to their root or  base form

#remove ---->suffix or prefix 


In [37]:
#The stemmed version of each word is printed alongside its original form.
#create a stemmer obj 
stemmer=PorterStemmer()
#stem some exmple words on txt
for word in filtered_txt:
    stemmed_word=stemmer.stem(word)
    print(f"{word}---->{stemmed_word}")

U.S.---->u.s.
President---->presid
Donald---->donald
Trump---->trump
Quietly---->quietli
Signs---->sign
Law---->law
To---->to
Allow---->allow
Warrantless---->warrantless
Searches---->search
In---->in
Parts---->part
Of---->of
VA---->va
DC---->dc
and---->and
MDBy---->mdbi
Aaron---->aaron
Kesel---->kesel
As---->as
former---->former
White---->white
House---->hous
chief---->chief
of---->of
staff---->staff
and---->and
current---->current
Chicago---->chicago
Mayor---->mayor
Rahm---->rahm
Emanuel---->emanuel
has---->ha
said---->said
“---->“
You---->you
never---->never
let---->let
a---->a
serious---->seriou
crisis---->crisi
go---->go
to---->to
waste.---->waste.
”---->”
While---->while
everyone---->everyon
was---->wa
worried---->worri
about---->about
the---->the
hurricanes---->hurrican
U.S.---->u.s.
President---->presid
Donald---->donald
Trump---->trump
signed---->sign
away---->away
the---->the
Fourth---->fourth
Amendment---->amend
buried---->buri
in---->in
a---->a
new---->new
bill---->bill
He--

# lemmatization

In [38]:
# Create a WordNet lemmatizer obj 
lemmatizer = WordNetLemmatizer()

# Apply lemmatization to each word in txt
lemmatized_txt = [lemmatizer.lemmatize(word) for word in filtered_txt]


In [39]:
# Print the original and lemmatized text
print("Original Text:")
print(txt)
print("\nText after lemmatization:")
print(lemmatized_txt)

Original Text:
U.S. President Donald Trump Quietly Signs Law To Allow Warrantless Searches In Parts Of VA, DC and MDBy Aaron Kesel
As former White House chief of staff and current Chicago Mayor Rahm Emanuel has said: “You never let a serious crisis go to waste.” While everyone was worried about the hurricanes, U.S. President Donald Trump signed away the Fourth Amendment buried in a new bill. He set a precedent by empowering the police state to allow warrantless searches in parts of VA, MD and D.C. near the Metro train system.
The government can now openly enter and search private property without a warrant in parts of Virginia, Maryland and D.C.
The House Joint Resolution 76 was signed into law on Tuesday, Aug. 22, by President Trump. The text is rather boring, starting off with: “Granting the consent and approval of Congress for the Commonwealth of Virginia, the State of Maryland, and the District of Columbia to enter into a compact relating to the establishment of the Washington Metr

# Regular Exp 


In [40]:
#searching and extracting 
pattern= r"\b[A-Za-z0-9._%+-]+@[A-Za-z0-9.-]+\.[A-Z|a-z]{2,}\b"
text=txt
emails = re.findall(pattern, text)



In [41]:
print("Email addresses:", emails)

Email addresses: []


In [42]:
# Matching a date in MM/DD/YYYY format
pattern = r"\b\d{2}/\d{2}/\d{4}\b"

text = txt
match = re.search(pattern, text)
if match:
    print("Found date:", match.group())
else:
    print("Date not found.")

Date not found.


In [43]:
# Extract all words
text=txt# text=''.join(txt)
words = re.findall(r'\b\w+\b', text)
print("Words:", words)

Words: ['U', 'S', 'President', 'Donald', 'Trump', 'Quietly', 'Signs', 'Law', 'To', 'Allow', 'Warrantless', 'Searches', 'In', 'Parts', 'Of', 'VA', 'DC', 'and', 'MDBy', 'Aaron', 'Kesel', 'As', 'former', 'White', 'House', 'chief', 'of', 'staff', 'and', 'current', 'Chicago', 'Mayor', 'Rahm', 'Emanuel', 'has', 'said', 'You', 'never', 'let', 'a', 'serious', 'crisis', 'go', 'to', 'waste', 'While', 'everyone', 'was', 'worried', 'about', 'the', 'hurricanes', 'U', 'S', 'President', 'Donald', 'Trump', 'signed', 'away', 'the', 'Fourth', 'Amendment', 'buried', 'in', 'a', 'new', 'bill', 'He', 'set', 'a', 'precedent', 'by', 'empowering', 'the', 'police', 'state', 'to', 'allow', 'warrantless', 'searches', 'in', 'parts', 'of', 'VA', 'MD', 'and', 'D', 'C', 'near', 'the', 'Metro', 'train', 'system', 'The', 'government', 'can', 'now', 'openly', 'enter', 'and', 'search', 'private', 'property', 'without', 'a', 'warrant', 'in', 'parts', 'of', 'Virginia', 'Maryland', 'and', 'D', 'C', 'The', 'House', 'Joint', 

In [44]:
# Extract all numbers
text=txt
numbers = re.findall(r'\b\d+\b', text)
print("Numbers:", numbers)

Numbers: ['76', '22', '2015', '29', '5', '18', '2017']


In [45]:
# Remove special characters
text=txt
clean_text = re.sub(r'[^a-zA-Z0-9\s]', '', text)
print("Clean Text:", clean_text)


Clean Text: US President Donald Trump Quietly Signs Law To Allow Warrantless Searches In Parts Of VA DC and MDBy Aaron Kesel
As former White House chief of staff and current Chicago Mayor Rahm Emanuel has said You never let a serious crisis go to waste While everyone was worried about the hurricanes US President Donald Trump signed away the Fourth Amendment buried in a new bill He set a precedent by empowering the police state to allow warrantless searches in parts of VA MD and DC near the Metro train system
The government can now openly enter and search private property without a warrant in parts of Virginia Maryland and DC
The House Joint Resolution 76 was signed into law on Tuesday Aug 22 by President Trump The text is rather boring starting off with Granting the consent and approval of Congress for the Commonwealth of Virginia the State of Maryland and the District of Columbia to enter into a compact relating to the establishment of the Washington Metrorail Safety Commission
CNN ex

In [46]:
txt

'U.S. President Donald Trump Quietly Signs Law To Allow Warrantless Searches In Parts Of VA, DC and MDBy Aaron Kesel\nAs former White House chief of staff and current Chicago Mayor Rahm Emanuel has said: “You never let a serious crisis go to waste.” While everyone was worried about the hurricanes, U.S. President Donald Trump signed away the Fourth Amendment buried in a new bill. He set a precedent by empowering the police state to allow warrantless searches in parts of VA, MD and D.C. near the Metro train system.\nThe government can now openly enter and search private property without a warrant in parts of Virginia, Maryland and D.C.\nThe House Joint Resolution 76 was signed into law on Tuesday, Aug. 22, by President Trump. The text is rather boring, starting off with: “Granting the consent and approval of Congress for the Commonwealth of Virginia, the State of Maryland, and the District of Columbia to enter into a compact relating to the establishment of the Washington Metrorail Safet

In [47]:
# Assuming txt is your actual text
text= txt

# Check if the text contains the word "example"
contains_Commons = re.search(r'\bCommons\b', text)

if contains_Commons:
    print("The text contains the word 'Commons'.")
else:
    print("The text does not contain the word 'Commons'.")

The text contains the word 'Commons'.


In [48]:
# Example text containing a URL
text=txt
# Regular expression pattern to match URLs
url_pattern = r'https?://\S+|www\.\S+'
# Remove URLs from the text
text_without_urls = re.sub(url_pattern, '', text)

print(text_without_urls)

U.S. President Donald Trump Quietly Signs Law To Allow Warrantless Searches In Parts Of VA, DC and MDBy Aaron Kesel
As former White House chief of staff and current Chicago Mayor Rahm Emanuel has said: “You never let a serious crisis go to waste.” While everyone was worried about the hurricanes, U.S. President Donald Trump signed away the Fourth Amendment buried in a new bill. He set a precedent by empowering the police state to allow warrantless searches in parts of VA, MD and D.C. near the Metro train system.
The government can now openly enter and search private property without a warrant in parts of Virginia, Maryland and D.C.
The House Joint Resolution 76 was signed into law on Tuesday, Aug. 22, by President Trump. The text is rather boring, starting off with: “Granting the consent and approval of Congress for the Commonwealth of Virginia, the State of Maryland, and the District of Columbia to enter into a compact relating to the establishment of the Washington Metrorail Safety Co

# Pos tagg

#Part-of-speech (POS) tagging is a natural language processing task that involves assigning a part of speech (such as noun, verb, adjective, etc.) to each word in a sentence 
. Libraries like NLTK and spaCy provide POS tagging functionalities.

In [49]:
import spacy
spacy.cli.download("en_core_web_sm")
# Load the English model in spaCy
nlp = spacy.load('en_core_web_sm')

#  text
text = txt

# Process the text using spaCy
doc = nlp(text)

# Extract POS tags for each token
pos_tags_spacy = [(token.text, token.pos_) for token in doc]

print(pos_tags_spacy)

✔ Download and installation successful
You can now load the package via spacy.load('en_core_web_sm')
[('U.S.', 'PROPN'), ('President', 'PROPN'), ('Donald', 'PROPN'), ('Trump', 'PROPN'), ('Quietly', 'ADV'), ('Signs', 'PROPN'), ('Law', 'PROPN'), ('To', 'PART'), ('Allow', 'VERB'), ('Warrantless', 'PROPN'), ('Searches', 'PROPN'), ('In', 'ADP'), ('Parts', 'PROPN'), ('Of', 'ADP'), ('VA', 'PROPN'), (',', 'PUNCT'), ('DC', 'PROPN'), ('and', 'CCONJ'), ('MDBy', 'PROPN'), ('Aaron', 'PROPN'), ('Kesel', 'PROPN'), ('\n', 'SPACE'), ('As', 'ADP'), ('former', 'ADJ'), ('White', 'PROPN'), ('House', 'PROPN'), ('chief', 'NOUN'), ('of', 'ADP'), ('staff', 'NOUN'), ('and', 'CCONJ'), ('current', 'ADJ'), ('Chicago', 'PROPN'), ('Mayor', 'PROPN'), ('Rahm', 'PROPN'), ('Emanuel', 'PROPN'), ('has', 'AUX'), ('said', 'VERB'), (':', 'PUNCT'), ('“', 'PUNCT'), ('You', 'PRON'), ('never', 'ADV'), ('let', 'VERB'), ('a', 'DET'), ('serious', 'ADJ'), ('crisis', 'NOUN'), ('go', 'VERB'), ('to', 'ADP'), ('waste', 'NOUN'), ('.', 'P

# feature extraction(Apply NLP techniques)



# Bag of word 

In [50]:
from sklearn.feature_extraction.text import TfidfTransformer
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.svm import SVC
from sklearn.naive_bayes import MultinomialNB
from sklearn.tree import DecisionTreeClassifier
from sklearn.metrics import accuracy_score

In [51]:
#filtered _txt ----> tokenization+remove stopwords and punctuation+LEmm
documents = filtered_txt

# Creating an instance of CountVectorizer
vectorizer = CountVectorizer()

# Fit and transform the documents
X = vectorizer.fit_transform(documents)

# Get the feature names (words/tokens)
feature_names = vectorizer.get_feature_names()

# Display the matrix of token counts
print("Matrix of token counts:")
print(X.toarray())

# Display feature names
print("\nFeature names (words/tokens):")
print(feature_names)

Matrix of token counts:
[[0 0 0 ... 0 0 0]
 [0 0 0 ... 0 0 0]
 [0 0 0 ... 0 0 0]
 ...
 [0 0 0 ... 0 0 0]
 [0 0 0 ... 0 0 0]
 [0 0 0 ... 0 0 0]]

Feature names (words/tokens):
['18', '2015', '2017', '22', '29', '76', 'aaron', 'able', 'about', 'according', 'across', 'activist', 'ad', 'adjacent', 'affect', 'after', 'against', 'agency', 'agents', 'all', 'allow', 'allowing', 'amash', 'amendment', 'americans', 'and', 'anti', 'any', 'anything', 'apparently', 'approval', 'are', 'area', 'article', 'as', 'at', 'attribution', 'aug', 'authority', 'away', 'based', 'be', 'been', 'bill', 'blevins', 'bloomberg', 'blueprint', 'bna', 'board', 'border', 'boring', 'bother', 'bothered', 'buried', 'by', 'called', 'can', 'carry', 'cause', 'chair', 'charge', 'chicago', 'chief', 'cnn', 'co', 'coinivore', 'columbia', 'commission', 'commons', 'commonwealth', 'compact', 'congress', 'congressman', 'consent', 'constitution', 'content', 'could', 'country', 'creative', 'credit', 'crisis', 'current', 'daunting', 'dc',

# TF-iDF Vectorizer

In [52]:
# Sample documents (preprocessed)
preprocessed_documents =filtered_txt

# Creating an instance of TfidfVectorizer
tfidf_vectorizer = TfidfVectorizer()

# Fit and transform the preprocessed documents
X_tfidf = tfidf_vectorizer.fit_transform(preprocessed_documents)

# Get the feature names (words/tokens)
tfidf_feature_names = tfidf_vectorizer.get_feature_names()

# Display the TF-IDF matrix
print("TF-IDF matrix:")
print(X_tfidf.toarray())

# Display feature names
print("\nFeature names (words/tokens):")
print(tfidf_feature_names)

TF-IDF matrix:
[[0. 0. 0. ... 0. 0. 0.]
 [0. 0. 0. ... 0. 0. 0.]
 [0. 0. 0. ... 0. 0. 0.]
 ...
 [0. 0. 0. ... 0. 0. 0.]
 [0. 0. 0. ... 0. 0. 0.]
 [0. 0. 0. ... 0. 0. 0.]]

Feature names (words/tokens):
['18', '2015', '2017', '22', '29', '76', 'aaron', 'able', 'about', 'according', 'across', 'activist', 'ad', 'adjacent', 'affect', 'after', 'against', 'agency', 'agents', 'all', 'allow', 'allowing', 'amash', 'amendment', 'americans', 'and', 'anti', 'any', 'anything', 'apparently', 'approval', 'are', 'area', 'article', 'as', 'at', 'attribution', 'aug', 'authority', 'away', 'based', 'be', 'been', 'bill', 'blevins', 'bloomberg', 'blueprint', 'bna', 'board', 'border', 'boring', 'bother', 'bothered', 'buried', 'by', 'called', 'can', 'carry', 'cause', 'chair', 'charge', 'chicago', 'chief', 'cnn', 'co', 'coinivore', 'columbia', 'commission', 'commons', 'commonwealth', 'compact', 'congress', 'congressman', 'consent', 'constitution', 'content', 'could', 'country', 'creative', 'credit', 'crisis', '

In [53]:
# define dependent and independent variable
#x------>Body ----> train + test
#y------>label --->train +test

x=df['Body'].values
y=df['Label'].values

In [54]:
#Splitting the dataset to training & test data
x_train, x_test, y_train, y_test = train_test_split(x, y, test_size = 0.2, stratify=y, random_state=2)

In [55]:
#vectorize variable 
# -------------converting the textual data to numerical data----------------
#vectorization = TfidfVectorizer()
#vectorization.fit(x)
#x=vectorization.transform(x)

In [56]:
# -------------converting the textual data to numerical data---------------
#1-Initialization of TfidfVectorizer:object without specifying any additional parameters
#2-Transforming Training and Testing Data:


vectorization = TfidfVectorizer()
xv_train=vectorization.fit_transform(x_train)
xv_test=vectorization.transform(x_test)

In [57]:
print(xv_train )

  (0, 37365)	0.04837467898996414
  (0, 31727)	0.136099171933921
  (0, 14439)	0.12768783390746158
  (0, 72)	0.04960374228156878
  (0, 3996)	0.031407273511677394
  (0, 8797)	0.1515793802342224
  (0, 28342)	0.02760925379609134
  (0, 38836)	0.14559350345060584
  (0, 2559)	0.16278983886677242
  (0, 7545)	0.10204845972966985
  (0, 29534)	0.082226223086551
  (0, 24705)	0.052624020733772287
  (0, 3113)	0.025090712284068565
  (0, 12739)	0.10684803187290831
  (0, 25873)	0.06646709257397837
  (0, 26719)	0.11438878462218507
  (0, 6077)	0.12683242615486798
  (0, 32687)	0.06799442357479817
  (0, 5091)	0.03899420321404772
  (0, 40032)	0.04291392387661348
  (0, 35083)	0.05922665578415478
  (0, 32576)	0.1827742331487321
  (0, 7112)	0.07270637521485944
  (0, 23735)	0.32557967773354485
  (0, 23785)	0.22514044646022083
  :	:
  (3206, 44473)	0.011416346191102631
  (3206, 43646)	0.014168677761137833
  (3206, 43737)	0.01913056428551464
  (3206, 14547)	0.01425306379890292
  (3206, 30579)	0.019872756816361598


# logistic regression


In [58]:
from sklearn.linear_model import LogisticRegression

In [59]:
LR=LogisticRegression()


In [61]:
LR.fit(xv_train,y_train)
y_pred = LR.predict(xv_test)


In [62]:
# accuracy score on the test data
testing_data_accuracy=LR.score(xv_test,y_pred)

In [63]:
print('Accuracy score of the testing data : ', testing_data_accuracy)

Accuracy score of the testing data :  1.0


#  ML classifier ( apply  ML classifier SVM, NB, DT, RF, etc.) 

In [345]:
classifiers = [
    ('SVM', SVC()),
    ('Naive Bayes', MultinomialNB()),]

In [346]:
# Train and evaluate each classifier
for name, classifier in classifiers:
    # Train the classifier
    classifier.fit(xv_train, y_train)


In [347]:
 y_pred = classifier.predict(xv_test)


In [348]:
accuracy = (y_test, y_pred)

In [349]:
print(accuracy)

(array([0, 0, 0, 0, 1, 1, 0, 0, 1, 1, 1, 0, 0, 0, 0, 1, 1, 1, 1, 1, 0, 1,
       1, 0, 1, 0, 1, 1, 1, 0, 0, 1, 1, 0, 0, 1, 1, 0, 0, 0, 0, 1, 0, 1,
       1, 0, 0, 1, 0, 1, 0, 1, 0, 0, 0, 1, 0, 0, 1, 1, 1, 1, 1, 1, 0, 0,
       0, 0, 0, 0, 1, 1, 1, 0, 1, 0, 1, 0, 1, 0, 0, 0, 1, 1, 0, 1, 0, 1,
       1, 1, 0, 0, 0, 0, 1, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 1, 0, 0, 0,
       1, 1, 1, 0, 0, 0, 0, 0, 0, 1, 0, 1, 0, 0, 1, 1, 0, 0, 1, 1, 1, 1,
       0, 0, 0, 0, 0, 1, 0, 1, 1, 0, 0, 0, 0, 0, 1, 0, 0, 1, 0, 1, 0, 0,
       0, 1, 1, 1, 0, 0, 1, 0, 1, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 1, 0, 1,
       0, 0, 1, 0, 1, 1, 0, 0, 0, 0, 1, 0, 0, 0, 0, 1, 0, 1, 0, 1, 0, 1,
       0, 1, 1, 1, 1, 0, 1, 0, 1, 1, 0, 0, 0, 0, 0, 0, 0, 0, 1, 1, 1, 0,
       1, 0, 1, 1, 0, 0, 0, 0, 0, 0, 1, 0, 1, 0, 0, 0, 1, 1, 0, 0, 0, 1,
       1, 0, 1, 1, 1, 1, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 1, 1, 1, 0, 0, 1,
       0, 1, 1, 1, 0, 0, 1, 0, 1, 1, 0, 1, 1, 0, 1, 1, 0, 1, 0, 1, 0, 0,
       1, 1, 0, 0, 0, 0, 1, 1, 0, 0, 1, 1, 0, 1, 0

In [350]:
svm_classifier = SVC(kernel='linear')

In [351]:
svm_classifier.fit(xv_train, y_train)

SVC(kernel='linear')

In [352]:
y_pred = svm_classifier.predict(xv_test)

In [354]:
accuracy = accuracy_score(y_test, y_pred)


In [355]:
print(f"SVM accuracy: {accuracy:.4f}")

SVM accuracy: 0.9888
